# The `ImageROI` class

The purpose of the `ImageROI` class is to provide an interface for setting the positions of punch-holes and to do some basic light-dark classification.

## Initialization
`ImageROI` is intended to be called downstream of `ImageSample`, either by providing the `ImageSample` instance directly or the image. In rare cases, it may not be desired to initialize an `ImageSample` instance, for example, if you are certain that the image you provide already is the sample region. In this case, you can provide either the image (pay attention to the correct type, the type depends on how you load the image) or a file path:

```python
from msi_workflow import ImageROI

roi = ImageROI(path_image_file=r'your\path\image_file.png')
```
or
```python
import numpy as np
from msi_workflow import ImageROI

img = np.random.random((100, 100))
roi = ImageROI(image=img, image_type='np')
```

And finally, let's cover the option oF providing the `ImageSample` instance. Assuming that you followed the previous chapter, you can use your saved instance:

In [1]:
from msi_workflow import ImageSample, ImageROI

%matplotlib inline

path_folder = r'C:\Users\Yannick Zander\Promotion\Test data'

sample = ImageSample.from_disk(path_folder)
roi = ImageROI.from_parent(sample)

print(roi)

obj_color: light
_hw: (0, 0)
_image_original: []
_image: []
path_folder: C:\Users\Yannick Zander\Promotion\Test data


Again the most convenient way is to use a `Project` instance. 

In [2]:
from msi_workflow import get_project

project = get_project(is_MSI=True, path_folder=path_folder, is_laminated=False)
project.set_image_roi()

No age span specified, falling back to more general method
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:04<00:00,  1.74it/s]


You can see a warning that the age span is not specified. This is because the `Project` class immediately executes all steps, but let's examine those steps in a bit more detail

## Classifying the image
In this step, the image is divided up into background, light, and dark pixels. Background pixels are inherited from the `ImageSample` object, if possible. Otherwise, the Otsu filter is called on the input image. Let's check out what that looks like

In [3]:
sample.load()
sample

obj_color: light
_hw: (8592, 24088)
_image_original: [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
_image: [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
path_folder: C:\Users\Yannick Zander\Promotion\Test data
_xywh_ROI: (4673, 1045, 13451, 4682)
_image_roi: []

In [4]:
import matplotlib.pyplot as plt
plt.imshow(roi.mask_foreground)
plt.show()

ValueError: zero-size array to reduction operation fmax which has no identity

We now are entering the area where you have to choose whether your samples are laminated or contain any other distinctive horizons because for samples homogenous in color we can omit steps that try to identify layers. In this example, no lamination can be seen, so we can disable that option using the `is_laminated` keyword